살짝 튜닝해본 모델에다가 카톡대화내역 불러오기 => 긍정부정 판다.

In [39]:
# !pip install konlpy

In [40]:
# 3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리 해보기로 하였다.
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file( 'test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')



In [41]:
import pandas as pd
#x,y값을 추출후 데이터프레임에 넣는다.
# X 값 추출 피쳐feature, y값 추출target 

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})


In [42]:
# df_train.iloc[:,0]
from konlpy.tag import Okt

#문장을 토큰화한것만 따로 저장
# tokenized_data =[]

# for i in df_train.iloc[:,0]:
#     tokenized_data.append(Okt().morphs( i ))

In [43]:
import pickle

#토근화된 데이터를 저장하였음
# with open('tokenized_data.pkl','wb') as f:
#     pickle.dump(tokenized_data,f)

#토큰화된 데이터 불러오기
# with open('tokenized_data.pkl','rb') as f:
#     tokenized_data_loaded = pickle.load(f)

# tokenized_data_loaded

In [44]:
#라벨별로 분리해본것
bad = df_train[df_train.iloc[:,1]==0]
good = df_train[df_train.iloc[:,1]==1]


df_train = df_train[::4]

from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

vectorizer=TfidfVectorizer(tokenizer=Okt().morphs)
X = vectorizer.fit_transform( df_train.iloc[:,0])
y = df_train.iloc[:,1]


In [45]:
df_train[df_train.iloc[:,1]==0]  #18678 
df_train[df_train.iloc[:,1]==1]  #18822
df_train.shape

(37500, 2)

In [46]:
# 토큰화하고 빈도수(tfidf)에 기반한 벡터표현이 생성된 X를 따로저장
from scipy.sparse import save_npz ,load_npz

#vectorizer.fit_transform한것을 희소행렬로로 저장
save_npz('Okt_37500개_vectorized_data.npz', X)

#저장된 희소행렬을 불러오기
# X_loaded = load_npz('Okt_37500개_vectorized_data.npz')

#이렇게 불러온 X_loaded는 바로 모델학습에 사용가능


In [19]:
import numpy as np
X_train_dense = np.expand_dims( X_loaded.toarray(), axis=-1 )
# X_train_dense.shape
# X_loaded.shape[1]
# X_loaded.toarray().shape[1]

In [22]:
#DNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization , Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    # input_layer,
    Input(shape=(X_train_dense.shape[1],)),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.6),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.6),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
optimizer= Adam(learning_rate=0.002)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 콜백 정의
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# model.fit(X.toarray(), y, epochs = 10, batch_size = 500, validation_split=0.2)  #0.8417
model.fit(
    X_loaded.toarray(), y, 
    epochs = 20, batch_size = 50, 
    validation_split=0.3,
    callbacks=[early_stopping]
    ) 

c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/20
525/525 [==============================] - 51s 90ms/step - loss: 0.5639 - accuracy: 0.7272 - val_loss: 0.4565 - val_accuracy: 0.8116
Epoch 2/20
525/525 [==============================] - 40s 76ms/step - loss: 0.3451 - accuracy: 0.8546 - val_loss: 0.3960 - val_accuracy: 0.8261
Epoch 3/20
525/525 [==============================] - 44s 84ms/step - loss: 0.2438 - accuracy: 0.9075 - val_loss: 0.4398 - val_accuracy: 0.8182
Epoch 4/20
525/525 [==============================] - 40s 76ms/step - loss: 0.1903 - accuracy: 0.9285 - val_loss: 0.4760 - val_accuracy: 0.8177
Epoch 5/20
525/525 [==============================] - 35s 68ms/step - loss: 0.1614 - accuracy: 0.9407 - val_loss: 0.5173 - val_accuracy: 0.8153


In [23]:
model.save('./학습시킨 모델만 모아놓은 폴더/3_31_DNN 튜닝모델.h5')

In [26]:
test = ['정말 별로 입니다. 이게뭐니']

# 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_test_sample = X_loaded.transform(test)
X_test_sample_dense = X_test_sample.toarray()

# 모델을 사용하여 예측 수행
predictions = model.predict(X_test_sample_dense)
print(predictions)

# 예측 결과 출력 (긍정: 1, 부정: 0)
print("예측 결과:", np.round( predictions ).astype(int))

AttributeError: transform not found

닉네임기준으로 
카톡대화내용을 기준으로 테스트 문장을 여러개 불러와서 예측 후 그 값의 비율을 계산 출력해준다.

In [2]:
# 카톡대화 불러와서 정제,(정규화코드)
import re

def get_from_txt(txt):
    f= open(txt,"r", encoding='utf-8')
    data = f.read() #읽어오기
    data2=data.split('\n') #줄별로 나눔 

    sentences=[]
    # for i in data2:
    #     if i.strip(): 
    #         if i[0]=='[':
    #             sentences.append(i)
    # sentences

    for line in data2:
        pattern = r'\[(.*?)\]\s+\[(.*?)\]\s+(.+)'
        match = re.match(pattern, line)
        if match:
            name = match.group(1)  # 첫 번째 대괄호 안의 단어 추출
            time = match.group(2)  # 두 번째 대괄호 안의 단어 추출
            content = match.group(3)  # 대괄호 뒤의 내용 추출
            # print(name, time, content)
            temp=[name,time,content]
            sentences.append(temp)
            
    return sentences

    # for i in sentences:
    #     print(i[2]) #i[0] 이름 i[1] 시간 i[2] 문장 


In [4]:
get_from_txt('sample.txt')

[['김찬란', '오후 10:01', '사진'],
 ['김찬란', '오후 9:38', 'https://youtube.com/watch?v=o1G7DWFFHso&feature=share'],
 ['김찬란', '오후 10:42', '사진'],
 ['김찬란', '오후 10:42', '사기치고다니는 한장규'],
 ['김하영', '오후 10:45', '이야'],
 ['김하영', '오후 10:46', '게임에서도 치네'],
 ['김찬란', '오후 10:46', '진짜 니말이 맞다 진짜 친구면 친구한테 민폐되기 싫어서 돈빌려달란 소리도 안한다'],
 ['김하영', '오전 12:25', '이모티콘'],
 ['김하영', '오전 12:35', '회식끝나고 집가는중'],
 ['김찬란', '오전 12:35', '음음 초밥집 회식은 어디서 하냐'],
 ['김하영', '오전 12:35', '우리 실장형네 직이 고기집해서 글로갔다'],
 ['김하영', '오전 12:36', '회식하면 맨날 돼지고기만 먹는다'],
 ['김찬란', '오전 12:36', 'ㅋㅋㅋㅋㅋㅋ케'],
 ['김하영', '오전 12:36', '이모티콘'],
 ['김찬란', '오전 12:36', '아 내수용 회식'],
 ['김찬란', '오전 12:36', '사진'],
 ['김하영', '오전 12:36', 'ㅋㅋㅋㅋ 그런거 아니고 애들이 여기가 맛있다고 여기서먹음'],
 ['김찬란', '오전 12:37', '고기 얘시하니까'],
 ['김찬란', '오전 12:37', '나도 고기가 먹고 싶군'],
 ['김하영', '오전 12:37', 'ㅋㅋㅋㅋ '],
 ['김찬란', '오전 12:37', '낼 사먹어야겠다'],
 ['김하영', '오전 12:37', '와라 엉아가 소고기 사줄께'],
 ['김찬란', '오전 12:37', '내가 사야지 씹새야 입벌려'],
 ['김찬란', '오전 12:37', '고기 들어간'],
 ['김찬란', '오전 12:38', '너는 내가 사주고 싶으니까 '],
 ['김찬란', '오전 12:38', '취직해서

In [ ]:
# 저장된 모델 불러오기
from tensorflow.keras.models import load_models
